In [17]:
# pip install cryptography

In [18]:
# pip install Crypto

In [19]:
import base64

from cryptography.hazmat.primitives import serialization,hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed25519 import \
        Ed25519PublicKey, Ed25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

# from Crypto.Cipher import AES

In [20]:
# pip install pycryptodome

In [21]:
# pip install pycryptodome==3.4.3

In [22]:
# from Crypto.Cipher import AES

In [23]:
pip install pycryptodomex 


Note: you may need to restart the kernel to use updated packages.


In [24]:
from Cryptodome.Cipher import AES 

In [25]:
def b64(msg):
    #base64 encoding helper function
    return base64.encodebytes(msg).decode('utf-8').strip()

In [26]:
def hkdf(inp,len):
    #using hashed message authenication code(HMAC) based key derivation function(KDF) to obtain the key from an input
    hkdf = HKDF(algorithm=hashes.SHA256(),length=len,salt=b'',info=b'',backend=default_backend())
    return hkdf.derive(inp)

In [27]:
class Bob(object):
    def __init__(self):
        #X25519 is an elliptical curve used to obtain cryptographic keys
        #IKb is the long term identity key of Bob
        self.IKb=X25519PrivateKey.generate()
        #SPKb is the signed pre key of Bob.The public key is uploaded to the server alongwith the signature using Bob's identity key,therefore proving Bob
        # has access to the private key
        self.SPKb=X25519PrivateKey.generate()
        #OPKb is the one time pre keys. Each ones public keys are published on the server.When wants to communicate with Bob it is fetched by him,then 
        #deleted from the server
        self.OPKb=X25519PrivateKey.generate()

    def x3dh(self,alice):
        #first DH key exchange between Bob's Signed pre key and Alice's Identity key.
        dh1 = self.SPKb.exchange(alice.IKa.public_key())
        #second DH key exchange between Bob's Identity Key and Alice's Ephemeral Key.
        #The first two DH key exchanges are for mutual authentication.
        dh2 = self.IKb.exchange(alice.EKa.public_key())
        #third DH key exchange between Bob's One time pre key and Alice's ephemeral key.
        dh3 = self.OPKb.exchange(alice.EKa.public_key())
        #forth DH key exchange between Bob's Signed pre key and Alice's ephemeral key.
        dh4 = self.SPKb.exchange(alice.EKa.public_key())

        #Creation of the shared key by concatenating the four DH keys and applying HKDF on it
        self.sk=hkdf(dh1+dh2+dh3+dh4,32)
        print(f'[Bob] shared key: {b64(self.sk)}')
        

In [28]:
class Alice(object):
    #description of all are the same as the ones for Bob
    def __init__(self):
        self.IKa=X25519PrivateKey.generate()
        self.EKa = X25519PrivateKey.generate()

    def x3dh(self,bob):
        dh1 = self.IKa.exchange(bob.SPKb.public_key())
        dh2 = self.EKa.exchange(bob.IKb.public_key())
        dh3 = self.EKa.exchange(bob.OPKb.public_key())
        dh4 = self.EKa.exchange(bob.SPKb.public_key())

        self.sk = hkdf(dh1+dh2+dh3+dh4,32)
        print(f'[Alice] shared key: {b64(self.sk)}')

In [29]:
alice = Alice()
bob = Bob()

alice.x3dh(bob)

bob.x3dh(alice)

[Alice] shared key: jHRJ7lagAAkniz7YBHYu4trXbRFMOaJPv8Jd785EBdI=
[Bob] shared key: jHRJ7lagAAkniz7YBHYu4trXbRFMOaJPv8Jd785EBdI=
